# Data Story

1. Questions
    - Count (appointments per patient)
    - Bar Plot (appointments per day of week)
    - Histogram (patients by age)
    - Comparison (percent of patients by gender vs percent of appointments by gender)
    - Scatterplot (hypertension vs diabetes)
    - Time Series Plot (appointments by date)
2. Insights
    - Correlations
    - Hypotheses
3. Narrative
    - Present in a story form, needs to flow
    - What trends/relationships would make it more complete?

In [1]:
# package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/kris/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kris/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kris/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Import clean DataFrames from 2.0-jkg-data-wrangling

In [2]:
# import pickle files
df_clean = pd.read_pickle('../data/interim/clean_df.pickle')
df_patients = pd.read_pickle('../data/interim/patients_df.pickle')
df_appointments = pd.read_pickle('../data/interim/appointments_df.pickle')

**Counts (Appointments per Patient)**

**Bar Plot (Appointments per Day of Week)**

**Histogram (Patients by Age)**

**Comparison (Percent of Patients by Gender vs Percent of Appointments by Gender)**

**Scatterplot (Hypertension vs Diabetes)**

**Time Series Plot (Appointments by Date)**